#### task

input: libraries (names, lat and long), time (for radius), census tract data frame (geoids, polygons)
to do: 
- (nested for loop) 
    - loop through libraries 
     - radii polygons -- put onto dataframe
        - for each ct loop for intersections
            if ct intersects
            - get intersection of the radii
            - get area of intersection
            - using area of current ct calculate percent
            - add ct and % coverage to a dictionary with as a key, value pair (ct, %)
            - on ct datframe col score = col score +=
      
output: overlapping census tracts and proportion covered for each radius (total coverage proportion for each census tract)

In [1]:
#imports

from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
import folium
from pyproj import Geod

In [2]:
branchInfo = gpd.read_file('../data/clean/branches.csv')
populationDemographics = pd.read_csv('../data/clean/population_demographics.csv')

In [3]:
api = MapboxAPI()

In [4]:
# setting up data frames
branchInfo = branchInfo.drop(columns = ['HOURS OF OPERATION', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE', 'WEBSITE','PERMANENTLY_CLOSED'])

censusTracts = populationDemographics[['geoid','geometry','qualifying name']]
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )
#censusTracts.head(5)

In [5]:
censusAreas = []
geod=Geod(ellps='WGS84')

for index, row in censusTracts.iterrows():
    poly_area, poly_perimeter = geod.geometry_area_perimeter(row['geometry'])
    poly_area = poly_area*-1
    censusAreas.append(poly_area)
censusTracts['calculated area'] = censusAreas

#censusTracts.head()

In [25]:
branchInfo = branchInfo.set_crs('EPSG:4326')
branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])
branchInfo.head()

dictionary = {}
censusTracts['score'] = 0
branchInfo['dictionary'] = ''
for i in range(branchInfo.shape[0]):
    api_response =  api.get_data((branchInfo.loc[i,'LATITUDE'],branchInfo.loc[i,'LONGITUDE']), 5)
    branchInfo.loc[i,'geometry'] = shape(api_response)
    oneBranch = gpd.GeoDataFrame(branchInfo.loc[i], geometry=branchInfo['geometry'], crs='EPSG: 4326')
    #change so this assigns the geometry column the long way##
    for index, row in censusTracts.iterrows():
      if ((oneBranch.intersection(row['geometry']))[0]).is_empty:
         empty=[]
         empty.append("none")
      else:
         overlapPolygon = (oneBranch.intersection(row['geometry'])[0])
         poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
         overlapArea = poly_area*-1
         propOverlap = overlapArea / row['calculated area']
         dictionary[row['geoid']] = propOverlap
    branchInfo.loc[i,'dictionary']= [dictionary] 
censusTracts.sort_values('score', ascending = False)

AttributeError: 'NoneType' object has no attribute 'is_empty'

In [21]:
#censusTracts.loc[index,'score'] = censusTracts.loc[index,'score'] + propOverlap
branchInfo

,BRANCH,LOCATION,geometry,LATITUDE,LONGITUDE,dictionary
0,Albany Park,"(41.97557881655979, -87.71361314512697)","POLYGON ((-87.71361 41.99322, -87.71461 41.992...",41.975579,-87.713613,"[{17031140800: 0.10910496440467443, 1703116050..."
1,Altgeld,"(41.65719847558056, -87.59883829075173)","POLYGON ((-87.59684 41.69523, -87.59691 41.693...",41.657198,-87.598838,"[{17031140800: 0.10910496440467443, 1703116050..."
2,Archer Heights,"(41.80110836194246, -87.72648385568911)","POLYGON ((-87.72348 41.81848, -87.72559 41.816...",41.801108,-87.726484,"[{17031140800: 0.10910496440467443, 1703116050..."
3,Austin,"(41.88917312206642, -87.7658203582574)","POLYGON ((-87.76482 41.90631, -87.76582 41.906...",41.889173,-87.765820,"[{17031140800: 0.10910496440467443, 1703116050..."
4,Austin-Irving,"(41.95312267684315, -87.77928489355646)","POLYGON ((-87.78829 41.96819, -87.78833 41.962...",41.953123,-87.779285,"[{17031140800: 0.10910496440467443, 1703116050..."
...,...,...,...,...,...,...
77,West Pullman,"(41.67790647382097, -87.6431683153105)","POLYGON ((-87.64217 41.70308, -87.64351 41.701...",41.677906,-87.643168,"[{17031140800: 0.10910496440467443, 1703116050..."
78,West Town,"(41.89594730478282, -87.66829305460979)","POLYGON ((-87.66729 41.91073, -87.66881 41.909...",41.895947,-87.668293,"[{17031140800: 0.10910496440467443, 1703116050..."
79,"Whitney M. Young, Jr.","(41.7510184591363, -87.61495065361933)","POLYGON ((-87.63095 41.78607, -87.63108 41.779...",41.751018,-87.614951,"[{17031140800: 0.10910496440467443, 1703116050..."
80,Woodson Regional Library,"(41.720694885749005, -87.64304817213312)","POLYGON ((-87.64305 41.74373, -87.64405 41.744...",41.720695,-87.643048,"[{17031140800: 0.10910496440467443, 1703116050..."
